In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

### Constants
This section initializes the piece_values and piece_square_tables, required by the different algorithms. <br>
This section is essential for any action including moves, pieces or boards. Which are all actions!

In [ ]:
# A score that is higher than any achievable score from an evaluation. Minimax and alphabeta start with this score but negative,
# so that the score that comes after is guaranteed to be the new best score (otherwise, a score that is not associated with
# any move would be the best score).
max_evaluation_score = 100_000_000

# The score for a move that creates a checkmate - such a move should always be chosen if on the winning side and should
# always be avoided if on the losing side.
evaluation_score_checkmate = 10_000_000

# The score for a move that creates a guaranteed win or loss as defined in the endgame tablebase - such a move should
# always be chosen if on the winning side and should always be avoided if on the losing side, except if a checkmate is also
# available. While a guaranteed win is theoretically just as good as a checkmate, a checkmate gets the win faster and
# is therefore preferable.
evaluation_score_endgame = 1_000_000

# The score for when a game ends in a draw. This is advantageous if a player is in a losing position and disadvantageous if they're
# in a winning position. In any case, it is less significant than a checkmate or a guaranteed win or loss.
evaluation_score_draw = 500_000

piece_values = {
    'p': 100,   # Pawns
    'n': 320,   # Knights
    'b': 330,   # Bishops
    'r': 500,   # Rooks
    'q': 900,   # Queen
    'k': 20000  # King
}

piece_square_tables = {
    'p': [ # Pawns
        [   0,   0,   0,   0,   0,   0,   0,   0 ],
        [  50,  50,  50,  50,  50,  50,  50,  50 ],
        [  10,  10,  20,  30,  30,  20,  10,  10 ],
        [   5,   5,  10,  25,  25,  10,   5,   5 ],
        [   0,   0,   0,  20,  20,   0,   0,   0 ],
        [   5,  -5, -10,   0,   0, -10,  -5,   5 ],
        [   5,  10,  10, -20, -20,  10,  10,   5 ],
        [   0,   0,   0,   0,   0,   0,   0,   0 ]
    ],

    'n': [ # Knights
        [ -50, -40, -30, -30, -30, -30, -40, -50 ],
        [ -40, -20,   0,   0,   0,   0, -20, -40 ],
        [ -30,   0,  10,  15,  15,  10,   0, -30 ],
        [ -30,   5,  15,  20,  20,  15,   5, -30 ],
        [ -30,   0,  15,  20,  20,  15,   0, -30 ],
        [ -30,   5,  10,  15,  15,  10,   5, -30 ],
        [ -40, -20,   0,   5,   5,   0, -20, -40 ],
        [ -50, -40, -30, -30, -30, -30, -40, -50 ]
    ],

    'b': [ # Bishops
        [ -20, -10, -10, -10, -10, -10, -10, -20 ],
        [ -10,   0,   0,   0,   0,   0,   0, -10 ],
        [ -10,   0,   5,  10,  10,   5,   0, -10 ],
        [ -10,   5,   5,  10,  10,   5,   5, -10 ],
        [ -10,   0,  10,  10,  10,  10,   0, -10 ],
        [ -10,  10,  10,  10,  10,  10,  10, -10 ],
        [ -10,   5,   0,   0,   0,   0,   5, -10 ],
        [ -20, -10, -10, -10, -10, -10, -10, -20 ]
    ],

    'r': [ # Rooks
        [   0,   0,   0,   0,   0,   0,   0,   0 ],
        [   5,  10,  10,  10,  10,  10,  10,   5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [   0,   0,   0,   5,   5,   0,   0,   0 ]
    ],

    'q': [ # Queen
        [ -20, -10, -10,  -5,  -5, -10, -10, -20 ],
        [ -10,   0,   0,   0,   0,   0,   0, -10 ],
        [ -10,   0,   5,   5,   5,   5,   0, -10 ],
        [  -5,   0,   5,   5,   5,   5,   0,  -5 ],
        [   0,   0,   5,   5,   5,   5,   0,  -5 ],
        [ -10,   5,   5,   5,   5,   5,   0, -10 ],
        [ -10,   0,   5,   0,   0,   0,   0, -10 ],
        [ -20, -10, -10,  -5,  -5, -10, -10, -20 ]
    ],

    'km': [ # King (middle game)
        [ -30, -40, -40, -50, -50, -40, -40, -30 ],
        [ -30, -40, -40, -50, -50, -40, -40, -30 ],
        [ -30, -40, -40, -50, -50, -40, -40, -30 ],
        [ -30, -40, -40, -50, -50, -40, -40, -30 ],
        [ -20, -30, -30, -40, -40, -30, -30, -20 ],
        [ -10, -20, -20, -20, -20, -20, -20, -10 ],
        [  20,  20,   0,   0,   0,   0,  20,  20 ],
        [  20,  30,  10,   0,   0,  10,  30,  20 ]
    ],

    'ke': [ # King (end game)
        [ -50, -40, -30, -20, -20, -30, -40, -50 ],
        [ -30, -20, -10,   0,   0, -10, -20, -30 ],
        [ -30, -10,  20,  30,  30,  20, -10, -30 ],
        [ -30, -10,  30,  40,  40,  30, -10, -30 ],
        [ -30, -10,  30,  40,  40,  30, -10, -30 ],
        [ -30, -10,  20,  30,  30,  20, -10, -30 ],
        [ -30, -30,   0,   0,   0,   0, -30, -30 ],
        [ -50, -30, -30, -30, -30, -30, -30, -50 ]
    ]
}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>